<a href="https://colab.research.google.com/github/deepthi-aiml/My-AI-ML-Projects/blob/main/Project_02_Dice_Roll_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import pandas as pd

In [ ]:
# Set style for better visualizations
plt.style.use('default')
sns.set_palette("husl")

class DiceRollSimulator:
    def __init__(self, dice_sides=6, num_dice=1):
        """
        Initialize the dice roll simulator

        Parameters:
        dice_sides (int): Number of sides on each die (default: 6)
        num_dice (int): Number of dice to roll (default: 1)
        """
        self.dice_sides = dice_sides
        self.num_dice = num_dice
        self.rolls_history = []

    def roll_dice(self, num_rolls=1000):
        """
        Simulate rolling dice multiple times

        Parameters:
        num_rolls (int): Number of times to roll the dice

        Returns:
        numpy array: Array of roll results
        """
        # Generate random rolls
        rolls = np.random.randint(1, self.dice_sides + 1, size=(num_rolls, self.num_dice))

        # If multiple dice, sum the results
        if self.num_dice > 1:
            results = np.sum(rolls, axis=1)
        else:
            results = rolls.flatten()

        self.rolls_history.extend(results)
        return results

    def theoretical_probability(self):
        """
        Calculate theoretical probability distribution

        Returns:
        dict: Theoretical probabilities for each possible outcome
        """
        if self.num_dice == 1:
            # For single die, uniform distribution
            outcomes = range(1, self.dice_sides + 1)
            prob = 1.0 / self.dice_sides
            return {outcome: prob for outcome in outcomes}
        else:
            # For multiple dice, calculate combinations
            from itertools import product

            # Generate all possible combinations
            all_combinations = product(range(1, self.dice_sides + 1), repeat=self.num_dice)
            sums = [sum(comb) for comb in all_combinations]

            # Count occurrences
            total_outcomes = self.dice_sides ** self.num_dice
            prob_dict = {}

            for outcome in range(self.num_dice, self.num_dice * self.dice_sides + 1):
                count = sums.count(outcome)
                prob_dict[outcome] = count / total_outcomes

            return prob_dict

    def empirical_probability(self, rolls):
        """
        Calculate empirical probability from actual rolls

        Parameters:
        rolls (array): Array of roll results

        Returns:
        dict: Empirical probabilities for each outcome
        """
        total_rolls = len(rolls)
        counter = Counter(rolls)

        return {outcome: count / total_rolls for outcome, count in counter.items()}


    def plot_distribution(self, rolls, title="Dice Roll Distribution"):
        """
        Plot both theoretical and empirical distributions

        Parameters:
        rolls (array): Array of roll results
        title (str): Plot title
        """
        # Calculate probabilities
        theoretical_probs = self.theoretical_probability()
        empirical_probs = self.empirical_probability(rolls)

        # Create subplots
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

        # Plot theoretical distribution
        outcomes_theoretical = list(theoretical_probs.keys())
        probs_theoretical = list(theoretical_probs.values())

        ax1.bar(outcomes_theoretical, probs_theoretical, alpha=0.7, color='skyblue', edgecolor='navy')
        ax1.set_xlabel('Outcome')
        ax1.set_ylabel('Probability')
        ax1.set_title('Theoretical Probability Distribution')
        ax1.grid(True, alpha=0.3)

        # Plot empirical distribution
        outcomes_empirical = list(empirical_probs.keys())
        probs_empirical = list(empirical_probs.values())

        ax2.bar(outcomes_empirical, probs_empirical, alpha=0.7, color='lightcoral', edgecolor='darkred')
        ax2.set_xlabel('Outcome')
        ax2.set_ylabel('Probability')
        ax2.set_title('Empirical Probability Distribution')
        ax2.grid(True, alpha=0.3)

        plt.suptitle(title, fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.show()

    def statistical_analysis(self, rolls):
        """
        Perform comprehensive statistical analysis

        Parameters:
        rolls (array): Array of roll results

        Returns:
        dict: Statistical measures
        """
        stats = {
            'Total Rolls': len(rolls),
            'Mean': np.mean(rolls),
            'Median': np.median(rolls),
            'Standard Deviation': np.std(rolls),
            'Variance': np.var(rolls),
            'Minimum': np.min(rolls),
            'Maximum': np.max(rolls),
            'Range': np.ptp(rolls)
        }

        return stats

    def convergence_analysis(self, rolls, step_size=100):
        """
        Analyze how empirical probability converges to theoretical

        Parameters:
        rolls (array): Array of roll results
        step_size (int): Step size for convergence analysis
        """
        theoretical_probs = self.theoretical_probability()
        target_outcome = list(theoretical_probs.keys())[len(theoretical_probs.keys()) // 2]  # Middle outcome

        empirical_probs_over_time = []
        theoretical_prob = theoretical_probs[target_outcome]

        for i in range(step_size, len(rolls) + 1, step_size):
            subset = rolls[:i]
            emp_prob = np.sum(subset == target_outcome) / len(subset)
            empirical_probs_over_time.append(emp_prob)

        # Plot convergence
        plt.figure(figsize=(10, 6))
        rolls_range = range(step_size, len(rolls) + 1, step_size)

        plt.plot(rolls_range, empirical_probs_over_time,
                label=f'Empirical P({target_outcome})', linewidth=2, marker='o')
        plt.axhline(y=theoretical_prob, color='red', linestyle='--',
                   label=f'Theoretical P({target_outcome}) = {theoretical_prob:.3f}')

        plt.xlabel('Number of Rolls')
        plt.ylabel('Probability')
        plt.title(f'Convergence of Empirical Probability to Theoretical\n(Outcome: {target_outcome})')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.show()


In [ ]:
def interactive_simulation():
    """
    Interactive function for custom simulations
    """
    print("\n🎯 INTERACTIVE DICE SIMULATION")
    print("=" * 40)

    try:
        dice_sides = int(input("Enter number of sides per die (default 6): ") or 6)
        num_dice = int(input("Enter number of dice (default 1): ") or 1)
        num_rolls = int(input("Enter number of rolls (default 1000): ") or 1000)

        simulator = DiceRollSimulator(dice_sides=dice_sides, num_dice=num_dice)
        rolls = simulator.roll_dice(num_rolls=num_rolls)

        print(f"\nSimulation Results:")
        print(f"Dice: {num_dice}d{dice_sides}")
        print(f"Rolls: {num_rolls}")

        # Display basic statistics
        stats = simulator.statistical_analysis(rolls)
        for key, value in stats.items():
            print(f"{key}: {value:.4f}")

        # Plot results
        title = f"Distribution of {num_dice}d{dice_sides} ({num_rolls} rolls)"
        simulator.plot_distribution(rolls, title)

        # Show most common outcomes
        counter = Counter(rolls)
        most_common = counter.most_common(5)
        print(f"\nMost Common Outcomes:")
        for outcome, count in most_common:
            probability = count / num_rolls
            print(f"Outcome {outcome}: {count} times ({probability:.3%})")

    except ValueError:
        print("Please enter valid numbers!")


In [ ]:
# Run the comprehensive analysis
if __name__ == "__main__":
    # Install required packages (Colab-friendly)
    try:
        import numpy as np
        import matplotlib.pyplot as plt
        import seaborn as sns
    except ImportError as e:
        print(f"Missing dependency: {e}")
        print("Please install required packages:")
        print("!pip install numpy matplotlib seaborn")

    # Run the analysis
    run_comprehensive_analysis()

    # Run interactive simulation
    interactive_simulation()

    print("\n" + "="*50)
    print("ANALYSIS COMPLETE! 🎯")
    print("Key Insights:")
    print("- Empirical probabilities converge to theoretical with more rolls")
    print("- Law of Large Numbers is clearly demonstrated")
    print("- Distribution shapes match expected probability theory")
    print("="*50)